# Dask DataFrame (2)

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы: 
* Макрушин С.В. Лекция "Dask DataFrame"
* https://docs.dask.org/en/latest/dataframe.html
* Jesse C. Daniel. Data Science with Python and Dask. 

* https://docs.dask.org/en/stable/generated/dask.dataframe.groupby.DataFrameGroupBy.apply.html
* https://docs.dask.org/en/stable/generated/dask.dataframe.Series.map_overlap.html#dask.dataframe.Series.map_overlap
* https://docs.dask.org/en/stable/_modules/dask/dataframe/rolling.html#Rolling.apply
* https://dask-sql.readthedocs.io/en/latest/quickstart.html

## Задачи для совместного разбора

In [1]:
import dask_sql
from dask_sql import Context

1. Загрузите данные о пользователях из архива каталога users. Посчитайте средний возраст каждого представленного пола.

In [2]:
import dask.dataframe as dd

ddf = dd.read_csv("users/*.csv")
ddf.head(2)

,id,name,age,gender
0,0,GBWAKSZR,57,female
1,1,JZFLVFWE,71,female


In [3]:
ddf.groupby("gender")["age"].mean().compute()

gender
female    48.303251
male      48.155142
Name: age, dtype: float64

In [4]:
ddf.groupby("gender")

2. Создайте bag на основе данных из файла users.txt. Преобразуйте его в dataframe и объедините с фреймом из задачи 1. 


3. Вычислите произведение каждой пары последовательно идущих возрастов людей.

4. Используя dask_sql, найдите топ-5 возрастов, имеющих наибольшее кол-во представителей.

In [ ]:
c = Context()

In [ ]:
c.create_table('users', ddf)

In [ ]:
c.sql('''
SELECT *
FROM users
''').head()

5. Создайте функцию rename_gender и сделайте выборку из таблицы с ее использованием при помощи dask_sql

## Лабораторная работа 13

__При решении данных задач не подразумевается использования циклов или генераторов Python в ходе работы с пакетами `numpy`, `pandas` и `dask`, если в задании не сказано обратного. Решения задач, в которых для обработки массивов `numpy`, структур `pandas` или структур `dask` используются явные циклы (без согласования с преподавателем), могут быть признаны некорректными и не засчитаны.__

В ходе выполнения все операции вычислений проводятся над `dask.DataFrame` и средствами пакета `dask`, если в задании не сказано обратного. Переход от коллекций `dask` возможен исключительно для демонстрации результата в конце решения задачи. Если в задаче используются результаты выполнения предыдущих задач, то подразумевается, что вы используете результаты в виде `dask.DataFrame` (то есть то, что было получено до вызова `compute`, а не после).

<p class="task" id="1"></p>

1\. В архиве `recipes_full.zip` находятся файлы, содержащие информацию об рецептах блюд. Загрузите данные из файлов этого архива в виде `dd.DataFrame` с названием `recipes`. Для каждого автора рассчитайте максимальное количество ингредиентов, которые используются в его рецептах.

In [1]:
import dask
import dask.dataframe as dd
import pandas as pd

In [2]:
recipes = dd.read_csv("recipes_full/recipes_full_*.csv", dtype={'minutes': 'float64', 'n_steps': 'float64'})

In [3]:
recipes.head()

,id,name,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,683970,vant ivoire mickies nothing,33.0,803776,2019-08-22,4.0,pat and gina neely and their family own and op...,9
1,1089012,kremsils mariposa baccala cookies class borage...,23.0,51579,2013-03-02,1.0,"a light, tasty and easy to put together chicke...",5
2,1428572,tania lander,0.0,68884,1980-11-09,1.0,a delicious melt in your mouth appetizer. for ...,5
3,1400250,heloise milli asher doogh zojirushi,24.0,678862,2018-04-29,3.0,delicious cream cheese and peach filled cresce...,1
4,387709,nutty chocolate chunk cookies,47.0,489552,2009-08-31,8.0,everyone loves these buttery cookies chock ful...,10


In [4]:
max_ingredients = recipes.groupby('contributor_id')['n_ingredients'].max().compute()

In [5]:
max_ingredients

contributor_id
27            10
1530          15
1531          13
1533          25
1534          20
              ..
2000111609     9
211616         8
254715         8
438612         7
795406        16
Name: n_ingredients, Length: 27926, dtype: int64

<p class="task" id="2"></p>

2\. Удалите строки, которые содержат пропуске в столбце `contributor_id` и приведите его тип к целочисленному. Для каждого `contributor_id` найдите топ-5 слов, которых этот пользователь наиболее часто использовал в названиях своих рецептов. Для разбиения на слова воспользуйтесь методом `.str.split`.

Выведите на экран статистику для пользователя с ID 1530.

In [6]:
import dask
import dask.dataframe as dd
import pandas

In [7]:
recipes = dd.read_csv("recipes_full/recipes_full_*.csv", dtype={'minutes': 'float64', 'n_steps': 'float64'})

In [8]:
recipes = recipes.dropna(subset=['contributor_id'])

In [9]:
# Преобразование столбца contributor_id в целочисленный тип
recipes['contributor_id'] = recipes['contributor_id'].astype(int)

In [10]:
def top_words(series):
    words = series.str.split(expand=True).stack().value_counts().head(5)
    return ', '.join(words.index)

In [11]:
top_words_by_contributor = recipes.groupby('contributor_id')['name'].apply(top_words).compute()

C:\Users\vladi\AppData\Local\Temp\ipykernel_21144\3365482400.py:1: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  top_words_by_contributor = recipes.groupby('contributor_id')['name'].apply(top_words).compute()


In [12]:
top_words_by_contributor[1530]

'chocolate, muffins, bread, and, whole'

<p class="task" id="3"></p>

3\. В архиве `stats.zip` находятся файлы, содержащие статистику по рецептам. Загрузите данные из файлов этого архива в виде `dd.DataFrame` с названием `stats`. Объедините две таблицы `recipes` и `stats`. Для каждого рецепта посчитайте среднее количество лайков за период, начиная с даты публикации рецепта и до 1 апреля 2023 года (включительно). Выведите на экран информацию о рецепте, имеющем максимальное значение этой величины.

In [13]:
from datetime import datetime

In [14]:
stats = dd.read_csv("stats/stats_*.csv")
recipes = dd.read_csv("recipes_full/recipes_full_*.csv", dtype={'minutes': 'float64','n_steps': 'float64'})

In [15]:
stats=stats.rename(columns = {'recipe_id':'id'})
stats.head()

,id,views,likes,verified
0,1979735,3331,552,False
1,283100,2168,2699,False
2,52604,2378,8444,False
3,1968356,3543,3097,True
4,1054c1,3164,4782,True


In [16]:
recipes['id'] = recipes['id'].astype('str')
stats['id'] = stats['id'].astype('str')

In [17]:
merged_data = dd.merge(recipes, stats, on='id')

In [18]:
merged_data.head(5)

,id,name,minutes,contributor_id,submitted,n_steps,description,n_ingredients,views,likes,verified
0,1579675,barkram mattie castillian metro page,21.0,452355,2019-04-17,2.0,NaN,1,1077,3462,True
1,800825,boles spoom instantaneous wifesaver frizzled s...,53.0,192951,2015-06-07,2.0,amounts are suggestions only. you can use let...,6,1901,3112,False
2,1507642,mimz,30.0,351811,2004-02-13,6.0,i found this recipe in the newspaper recently ...,8,8410,9009,False
3,1980534,raise gunkanmaki pomegrante cda shaky twilight...,19.0,89831,2014-10-06,5.0,"i am a huge bobby flay fan, so i found one of ...",8,2961,2071,True
4,810791,kima neeley 2014 rise revenge vegetarians menn...,15.0,143318,2014-03-07,2.0,"get out a big mug, get out the cappuccino make...",3,1065,8267,False


In [20]:
merged_data['submitted'] = dd.to_datetime(merged_data['submitted'])

In [21]:
filtered_data = merged_data[merged_data['submitted'] <= datetime(2023, 4, 1)]

In [22]:
average_likes = merged_data.groupby('id')['likes'].mean()

In [23]:
recipe_with_max_likes = merged_data[merged_data['id'] == average_likes.idxmax()]

In [24]:
recipe_with_max_likes.compute()

,id,name,minutes,contributor_id,submitted,n_steps,description,n_ingredients,views,likes,verified
204048,1033737,flode kahwa roschetzkys lasagna gerhard mcclel...,10.0,58892,1988-07-14,2.0,i came up with this when i was among the many ...,6,1693,9999,False


<p class="task" id="4"></p>

4\. Используя `dask_sql`, выберите из таблицы `recipes` id, название и описание тех рецептов, которые были добавлены в 2010 году. Выведите полученный результат на экран в виде `pd.DataFrame`. Выведите на экран количество  рецептов, которые были добавлены в 2010 году.

In [25]:
from dask_sql import Context

recipes = dd.read_csv("recipes_full/recipes_full_*.csv", dtype={'minutes': 'float64','n_steps': 'float64'})

In [26]:
# Создание объекта Dask SQL
c = Context()
c.create_table("recipes", recipes)

# Выбрать необходимые столбцы и отфильтровать рецепты, добавленные в 2010 году
recipes_2010 = """
    SELECT id, name, description
    FROM recipes
    WHERE EXTRACT(YEAR FROM submitted) = 2010
"""

# Преобразовать полученный результат в объект pd.DataFrame и вывести его на экран
res = c.sql(recipes_2010)

In [27]:
res.compute()

,id,name,description
8,818815,beirox dalmation gittle rougeux,my father is from iran and this is the way he ...
379,380095,shabbos sourest includes chevops schelvisch en...,"i have made several different chili recipes, a..."
381,1006702,stitch frisch roses oranged levana,"spinach, blueberries, and blue cheese in a ras..."
388,1519209,gingernuts anconetana reuben,this recipe is a take on my recipe #459346 usi...
407,438167,deep chocolate pistachio brownies,found in family circle. these have such a nic...
...,...,...,...
278701,1637133,nouvelle allegria,a great pumpkin pie recipe that is easy to mak...
278745,420691,tilapia with a creamy shrimp crab white wine...,"this is a really elegant dish, but really simp..."
278783,795632,sayonara,a delicious way to serve sweet potatoes from d...
278864,1332531,fashoulakia bisquickie awesomesauce ukrainian ...,stolen from http://aroundthetableri.blogspot.c...


In [28]:
len(res.compute())

50839

<p class="task" id="5"></p>

5\. Создайте функцию `get_recipe_types`, которая определяет сложность рецепта по следующему правилу: если кол-во ингредиентов не больше 5, то рецепт простой ("easy"); если не кол-во ингредиентов не больше 15, то рецепт средней сложности ("medium"); иначе рецепт является сложным ("hard"). 

```python
def get_type(ingredients: int) -> str:
    pass
```

Зарегистрируйте эту функцию для применения в SQL-запросе.

In [29]:
import dask.dataframe as dd
import numpy as np
from dask_sql import Context

In [30]:
recipes = dd.read_csv("recipes_full/recipes_full_*.csv", dtype={'minutes': 'float64','n_steps': 'float64'})
recipes['n_ingredients'] = recipes['n_ingredients'].astype(int)

In [31]:
def get_type(ingredients:int) -> str:
    if ingredients <= 5:
        return "easy"
    elif ingredients <= 15:
        return "medium"
    else:
        return "hard"

In [32]:
recipes['difficulty'] = recipes['n_ingredients'].apply(get_type, meta=('difficulty', 'object'))

In [33]:
recipes.head(5)

,id,name,minutes,contributor_id,submitted,n_steps,description,n_ingredients,difficulty
0,683970,vant ivoire mickies nothing,33.0,803776,2019-08-22,4.0,pat and gina neely and their family own and op...,9,medium
1,1089012,kremsils mariposa baccala cookies class borage...,23.0,51579,2013-03-02,1.0,"a light, tasty and easy to put together chicke...",5,easy
2,1428572,tania lander,0.0,68884,1980-11-09,1.0,a delicious melt in your mouth appetizer. for ...,5,easy
3,1400250,heloise milli asher doogh zojirushi,24.0,678862,2018-04-29,3.0,delicious cream cheese and peach filled cresce...,1,easy
4,387709,nutty chocolate chunk cookies,47.0,489552,2009-08-31,8.0,everyone loves these buttery cookies chock ful...,10,medium


In [34]:
recipes = dd.read_csv("recipes_full/recipes_full_*.csv", dtype={'minutes': 'float64','n_steps': 'float64', 'n_ingredients':'int'})

In [35]:
c = Context()
c.create_table("recipes", recipes)

В данном варианте выполнения второй части задания функция не соответствует той структуре, которую вы задали (в параметр передается не int)

In [36]:
def get_type1(ingredients):
    return ingredients.apply(lambda x: "easy" if x <= 5 else "medium" if x <= 15 else "hard")

In [37]:
c.register_function(
    get_type1,
    "gt1",
    [("n_ingredients", int)], # аргумент и его тип
    str # тип возвращаемого значения
)

In [38]:
query = "SELECT *, gt1(n_ingredients) AS difficulty FROM recipes"

result = c.sql(query)
result.compute()

C:\Users\vladi\anaconda3\lib\site-packages\dask\dataframe\core.py:4139: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('n_ingredients', 'object'))

  warnings.warn(meta_warning(meta))


,id,name,minutes,contributor_id,submitted,n_steps,description,n_ingredients,difficulty
0,683970,vant ivoire mickies nothing,33.0,803776,2019-08-22,4.0,pat and gina neely and their family own and op...,9,medium
1,1089012,kremsils mariposa baccala cookies class borage...,23.0,51579,2013-03-02,1.0,"a light, tasty and easy to put together chicke...",5,easy
2,1428572,tania lander,0.0,68884,1980-11-09,1.0,a delicious melt in your mouth appetizer. for ...,5,easy
3,1400250,heloise milli asher doogh zojirushi,24.0,678862,2018-04-29,3.0,delicious cream cheese and peach filled cresce...,1,easy
4,387709,nutty chocolate chunk cookies,47.0,489552,2009-08-31,8.0,everyone loves these buttery cookies chock ful...,10,medium
...,...,...,...,...,...,...,...,...,...
278949,1029131,tuti waffle snackies steakhouse,19.0,171345,1973-10-18,4.0,"according to a providence journal article, ama...",4,easy
278950,1700703,noelias cheats crocante fleisch zitumbuwa,1.0,30228,2007-07-01,6.0,if possible sauté the onions and garlic in abo...,1,easy
278951,1910650,rubbed restuffed pelmeni bedouin flavourful,60.0,591905,2009-09-26,3.0,another great recipe to add to the growing swe...,2,easy
278952,713836,stems polpettine peezi,NaN,357389,2003-09-30,4.0,adapted from top secret recipes. love this!,9,medium


В данном варианте выполнения второй части задания функция соответствует той структуре, которую вы задали, но код выдает ошибку при ывполнении sql запроса

In [39]:
recipes = dd.read_csv("recipes_full/recipes_full_*.csv", dtype={'minutes': 'float64','n_steps': 'float64', 'n_ingredients':'int'})

In [40]:
c = Context()
c.create_table("recipes", recipes)

In [41]:
c.register_function(
    get_type,
    "gt",
    [("n_ingredients", int)], # аргумент и его тип
    str # тип возвращаемого значения
)

In [42]:
c.sql('''
SELECT *, gt(n_ingredients) AS difficulty
FROM recipes
''').compute()

ValueError: The truth value of a Series is ambiguous. Use a.any() or a.all().

<p class="task" id="6"></p>

6\. Используя `dask_sql`, посчитайте количество рецептов с группировкой по их типу (простой, средний или сложный). Для определения типа рецепта в SQL-запросе воспользуйтесь функцией, зарегистрированной в предыдущем задании. Выведите на экран в виде серии с индексами "easy", "medium" и "hard". 

In [43]:
recipes = dd.read_csv("recipes_full/recipes_full_*.csv", dtype={'minutes': 'float64','n_steps': 'float64', 'n_ingredients':'int'})

In [44]:
c = Context()
c.create_table("recipes", recipes)

In [45]:
c.register_function(
    get_type1,
    "gt1",
    [("n_ingredients", int)], # аргумент и его тип
    str # тип возвращаемого значения
)

In [46]:
query = "SELECT gt1(n_ingredients) AS difficulty, COUNT(*) AS count FROM recipes GROUP BY gt1(n_ingredients)"

result = c.sql(query)
result.compute()

RuntimeError: DataFusionError(SchemaError(FieldNotFound { field: Column { relation: Some("recipes"), name: "n_ingredients" }, valid_fields: [Column { relation: None, name: "gt1(recipes.n_ingredients)" }, Column { relation: None, name: "COUNT(UInt8(1))" }] }))

<p class="task" id="7"></p>

7\. Используя `dask_sql`, объедините таблицы и посчитайте и выведите на экран, сколько рецептов являются верифицированными и не верифицированными. Если информация о верификации рецепта отсутствует, то считайте его не верифицированным. Вся логика подсчета значений должна быть реализована в виде запроса на языке SQL.

<p class="task" id="8"></p>

8\. В файлах архива `site_logs.zip` находятся логи системы. Считайте эти логи в виде `dask.bag`, выделите их них дату, время, id службы, уровень лога и сообщение. Объедините дату и время в одну строку в формате "ГГГГММДД ЧЧММСС". В итоге каждый элемент `Bag` должен представлять собой словарь с ключами `datetime`, `service_id`, `msg`, `level`.

Преобразуйте `dask.bag` в `dask.dataframe`, предварительно оставив только логи уровня "INFO". Подтвердите корректность решения, выведя на экран уникальные значения столбца `level` полученной таблицы.

```
date(ymd)   time(HMS)   service_id level   msg
081110      103041      34         INFO    dfs.FSNamesystem: BLOCK* NameSystem.delete: ...
```

<p class="task" id="9"></p>

9\. Добавьте в таблицу из предыдущего задания столбец `is_delete_op`, который содержит флаг: является ли данная операция удалением чего-либо или нет. Для выяснения этого факта проанализизируйте сообщение лога. 

Для каждого лога подсчитайте, сколько из записанных перед ним последних 100 логов являются операциями удаления. Для вычислений используйте метод `map_overlap`. Посчитайте и выведите на экран, сколько существует подпоследовательностей длины 100, состоящих не менее чем из 75 сообщений, содержащих операции удаления.

<p class="task" id="10"></p>

10\. Сгруппируйте таблицу с логами по часу, в котором был оставлен лог. Под часом имеется в виду час конкретного дня, то связь связка год-месяц-день-час.  Посчитайте, сколько логов было записано за каждый час. Визуализируйте полученную динамику. Добавьте подписи засечек, подписи осей и название рисунка.